In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

In [16]:
#---------------------Data Cleaning/Transformation-----------------------------------------
#read csv to dataframe
weather_df = pd.read_csv('./data/noaa_weather_data.csv', index_col='DATE')
#rename and filter columns which can be useful
core_weather_df = weather_df[['NAME','PRCP', 'SNOW','SNWD', 'TMAX','TMIN']].copy()
core_weather_df.columns = ['name','precip', 'snow', 'snow_depth', 'temp_max','temp_min']
#fill missing data points
core_weather_df['precip'] = core_weather_df['precip'].fillna(0)
core_weather_df['snow'] = core_weather_df['snow'].fillna(0)
core_weather_df['snow_depth'] = core_weather_df['snow_depth'].fillna(0)
#map the name to a number variable to be used as a predictor
core_weather_df['name_num'] = pd.factorize(core_weather_df['name'])[0]
#'ffill' ('forward fill') fills the value in with the value from the previous date this will apply to the temp_min/max columns since the others have already been filled in
core_weather_df = core_weather_df.fillna(method='ffill')
#change date index to a datetime data type
core_weather_df.index = pd.to_datetime(core_weather_df.index)
#make a visual representation of the data and remove cities that have large chunks of missing data
core_weather_df = core_weather_df[core_weather_df.name.isin(['BEMIDJI, MN US', 'SHARJAH INTER. AIRP, AE']) == False]
# core_weather_df.groupby(by='name').plot()
#create a column 'target_temp_max' by shifting all values in the temp_max column back a day...creates a column based on temperatures from tomorrows temps
core_weather_df['target_temp_max_day_1'] = core_weather_df.groupby('name')['temp_max'].shift(-1)
core_weather_df['target_temp_max_day_2'] = core_weather_df.groupby('name')['target_temp_max_day_1'].shift(-1)
core_weather_df['target_temp_max_day_3'] = core_weather_df.groupby('name')['target_temp_max_day_2'].shift(-1)
core_weather_df['target_temp_max_day_4'] = core_weather_df.groupby('name')['target_temp_max_day_3'].shift(-1)
core_weather_df['target_temp_max_day_5'] = core_weather_df.groupby('name')['target_temp_max_day_4'].shift(-1)
core_weather_df['target_temp_max_day_6'] = core_weather_df.groupby('name')['target_temp_max_day_5'].shift(-1)
core_weather_df['target_temp_max_day_7'] = core_weather_df.groupby('name')['target_temp_max_day_6'].shift(-1)
core_weather_df['target_temp_max_day_8'] = core_weather_df.groupby('name')['target_temp_max_day_7'].shift(-1)
core_weather_df['target_temp_max_day_9'] = core_weather_df.groupby('name')['target_temp_max_day_8'].shift(-1)
core_weather_df['target_temp_max_day_10'] = core_weather_df.groupby('name')['target_temp_max_day_9'].shift(-1)
#remove the last row since its a NaN value cuz that value would be in the future
core_weather_df = core_weather_df.dropna()

In [37]:
#-----------------create/test data model--------------------------------
#using Ridge regression to minimize overfitting
regression = Ridge(alpha=.1)
predictors = ['precip','snow','snow_depth','temp_max','temp_min']
#training set includes all data before and including dec 12th 2021, this is teh set that is used to train the model to predict future data
training_set = core_weather_df.loc[:'2021-12-31']
#test set includes all data after and including jan 1st 2022
test_set = core_weather_df.loc['2022-01-01':]
test_set_shifted = test_set.shift(365, freq='D')
test_set2= test_set_shifted.loc['2023-01-01':'2023-12-31']

#train the model based on the predictors
regression.fit(training_set[predictors], training_set['target_temp_max_day_1'])
predictions = regression.predict(test_set2[predictors])

combined_df = pd.concat([test_set2[['name']], pd.Series(predictions, index = test_set2.index)], axis=1)
combined_df.columns = ['name','Predicted_temp']

df = pd.DataFrame(predictions, columns=['Predicted_temp'])
core_weather_df.name.unique()
display(combined_df.loc[combined_df['name'] == 'PORTLAND INTERNATIONAL AIRPORT, OR US'])
display


,name,Predicted_temp
DATE,,
2023-01-01,"PORTLAND INTERNATIONAL AIRPORT, OR US",36.544817
2023-01-02,"PORTLAND INTERNATIONAL AIRPORT, OR US",49.337245
2023-01-03,"PORTLAND INTERNATIONAL AIRPORT, OR US",48.294390
2023-01-04,"PORTLAND INTERNATIONAL AIRPORT, OR US",48.031114
2023-01-05,"PORTLAND INTERNATIONAL AIRPORT, OR US",43.994891
...,...,...
2023-12-27,"PORTLAND INTERNATIONAL AIRPORT, OR US",54.659743
2023-12-28,"PORTLAND INTERNATIONAL AIRPORT, OR US",51.300502
2023-12-29,"PORTLAND INTERNATIONAL AIRPORT, OR US",47.165378


<function IPython.core.display.display(*objs, include=None, exclude=None, metadata=None, transient=None, display_id=None, **kwargs)>

In [18]:
test_set2 = test_set.shift(365, freq='D')
display(test_set.head())
display(test_set2.head())

,name,precip,snow,snow_depth,temp_max,temp_min,name_num,target_temp_max_day_1,target_temp_max_day_2,target_temp_max_day_3,target_temp_max_day_4,target_temp_max_day_5,target_temp_max_day_6,target_temp_max_day_7,target_temp_max_day_8,target_temp_max_day_9,target_temp_max_day_10
DATE,,,,,,,,,,,,,,,,,
2022-01-01,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.1,33.9,-7.0,-17.0,0,-16.0,-14.0,-19.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0
2022-01-02,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.0,33.1,-16.0,-47.0,0,-14.0,-19.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0
2022-01-03,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.0,33.1,-14.0,-45.0,0,-19.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0,21.0
2022-01-04,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.0,33.1,-19.0,-37.0,0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0,21.0,20.0
2022-01-05,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.0,33.1,-26.0,-38.0,0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0,21.0,20.0,7.0


,name,precip,snow,snow_depth,temp_max,temp_min,name_num,target_temp_max_day_1,target_temp_max_day_2,target_temp_max_day_3,target_temp_max_day_4,target_temp_max_day_5,target_temp_max_day_6,target_temp_max_day_7,target_temp_max_day_8,target_temp_max_day_9,target_temp_max_day_10
DATE,,,,,,,,,,,,,,,,,
2023-01-01,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.1,33.9,-7.0,-17.0,0,-16.0,-14.0,-19.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0
2023-01-02,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.0,33.1,-16.0,-47.0,0,-14.0,-19.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0
2023-01-03,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.0,33.1,-14.0,-45.0,0,-19.0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0,21.0
2023-01-04,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.0,33.1,-19.0,-37.0,0,-26.0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0,21.0,20.0
2023-01-05,"FAIRBANKS INTERNATIONAL AIRPORT, AK US",0.0,0.0,33.1,-26.0,-38.0,0,-6.0,-26.0,-30.0,2.0,3.0,7.0,12.0,21.0,20.0,7.0
